In [42]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import cv2
from PIL import Image, ImageTk
import os
import random
import csv


class ImageAnnotator:
    """
    A class for annotating images with bounding boxes.

    Attributes:
    - root: The root Tkinter window.
    - files_frame: The frame for the files section.
    - files_label: The label for the files section.
    - files_listbox: The listbox to display the image files.
    - load_button: The button to load image files.
    - delete_button: The button to delete an image file.
    - class_frame: The frame for the classes section.
    - class_label: The label for the classes section.
    - class_listbox: The listbox to display the available classes.
    - add_class_button: The button to add a new class.
    - delete_class_button: The button to delete a class.
    - save_button: The button to save annotations.
    - images: A list of image file paths.
    - current_image_index: The index of the currently selected image.
    - current_image: The currently selected image.
    - bbox_start: The starting point of a bounding box.
    - bbox_list: A dictionary to store bounding boxes for each image.
    - selected_class: The currently selected class.
    - classes: A dictionary to store class names and their corresponding colors.
    - canvas: The canvas to display the image and annotations.
    - annotations_dict: A dictionary to store annotations for each image.
    """

    def __init__(self, root):
        """
        Initializes the ImageAnnotator class.

        Parameters:
        - root: The root Tkinter window.
        """
        self.root = root
        self.root.title("Image Annotator")

        # ... rest of the code ...
class ImageAnnotator:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotator")

        # Create a frame for the files section
        self.files_frame = tk.Frame(root, bg="white")
        self.files_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create a label for the files section
        self.files_label = tk.Label(self.files_frame, text="Image Files", font=("Helvetica", 12), bg="white")
        self.files_label.pack(side=tk.TOP, pady=(0, 5))

        # Create a listbox to display the image files
        self.files_listbox = tk.Listbox(self.files_frame, width=40, height=20, bg="#f0f0f0")
        self.files_listbox.pack(side=tk.TOP, fill=tk.BOTH, padx=(0, 5), pady=(0, 5))
        self.files_listbox.bind('<<ListboxSelect>>', self.load_image)

        # Create buttons for loading and deleting image files
        self.load_button = tk.Button(self.files_frame, text="Load Images", command=self.load_images)
        self.load_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        self.delete_button = tk.Button(self.files_frame, text="Delete Image", command=self.delete_image)
        self.delete_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        # Create a frame for the classes section
        self.class_frame = tk.Frame(root, bg="white")
        self.class_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create a label for the classes section
        self.class_label = tk.Label(self.class_frame, text="Annotation Classes", font=("Helvetica", 12), bg="white")
        self.class_label.pack(side=tk.TOP, pady=(0, 5))

        # Create a listbox to display the available classes
        self.class_listbox = tk.Listbox(self.class_frame, width=20, height=10, bg="#f0f0f0")
        self.class_listbox.pack(side=tk.TOP, fill=tk.BOTH, padx=(0, 5), pady=(0, 5))
        self.class_listbox.bind('<<ListboxSelect>>', self.select_class)

        # Create buttons for adding and deleting classes
        self.add_class_button = tk.Button(self.class_frame, text="Add Class", command=self.add_class)
        self.add_class_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        self.delete_class_button = tk.Button(self.class_frame, text="Delete Class", command=self.delete_class)
        self.delete_class_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        # Create a button to save annotations
        self.save_button = tk.Button(root, text="Save Annotations", command=self.save_annotations)
        self.save_button.pack(side=tk.BOTTOM, pady=10)

        # Initialize attributes
        self.images = []
        self.current_image_index = None
        self.current_image = None
        self.bbox_start = None
        self.bbox_list = {}
        self.selected_class = None
        self.classes = {}

        # Create a canvas to display the image and annotations
        self.canvas = tk.Canvas(self.root, bg="white", highlightbackground="gray", highlightthickness=1)
        self.canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, padx=10, pady=10, expand=True)
        self.canvas.bind("<Button-1>", self.start_bbox)
        self.canvas.bind("<B1-Motion>", self.draw_bbox)
        self.canvas.bind("<ButtonRelease-1>", self.end_bbox)
        self.annotations_dict = {}

    def load_images(self):
        file_paths = filedialog.askopenfilenames(filetypes=[("Image files", "*.jpg; *.jpeg; *.png")])
        if file_paths:
            self.images = list(file_paths)
            self.files_listbox.delete(0, tk.END)  # Clear previous entries
            for image_path in self.images:
                self.files_listbox.insert(tk.END, os.path.basename(image_path))
            messagebox.showinfo("Images Loaded", "Image files loaded successfully.")
            self.clear_annotations()  # Clear annotations when loading new images

    def delete_image(self):
        selected_index = self.files_listbox.curselection()
        if selected_index:
            del self.images[selected_index[0]]
            self.files_listbox.delete(selected_index[0])
            messagebox.showinfo("Image Deleted", "Image file deleted successfully.")

    def load_image(self, event):
        selected_index = self.files_listbox.curselection()
        if selected_index:
            self.clear_annotations()  # Clear annotations when loading a new image
            self.current_image_index = selected_index[0]
            self.current_image = cv2.imread(self.images[self.current_image_index])
            self.display_image()

    def add_class(self):
        new_class = simpledialog.askstring("Add Class", "Enter new class name:")
        if new_class:
            self.class_listbox.insert(tk.END, new_class)
            self.classes[new_class] = '#' + "%06x" % random.randint(0, 0xFFFFFF)
            messagebox.showinfo("Class Added", "New class added successfully.")

    def delete_class(self):
        selected_index = self.class_listbox.curselection()
        if selected_index:
            selected_class = self.class_listbox.get(selected_index[0])
            del self.classes[selected_class]
            self.class_listbox.delete(selected_index[0])
            messagebox.showinfo("Class Deleted", "Class deleted successfully.")

    def select_class(self, event):
        selected_index = self.class_listbox.curselection()
        if selected_index:
            self.selected_class = self.class_listbox.get(selected_index[0])

    def start_bbox(self, event):
        if self.selected_class and self.current_image is not None:
            self.bbox_start = (event.x, event.y)

    def draw_bbox(self, event):
        if self.bbox_start is not None:
            x0, y0 = self.bbox_start
            x1, y1 = (event.x, event.y)
            self.canvas.delete("bbox")  # Delete previous bounding boxes
            # Draw all existing bounding boxes
            for bbox, cls in self.bbox_list.get(self.images[self.current_image_index], []):
                self.canvas.create_rectangle(*bbox, outline=self.classes[cls], tags="bbox")
            # Draw the current bounding box
            self.canvas.create_rectangle(x0, y0, x1, y1, outline=self.classes[self.selected_class], tags="bbox")

    def end_bbox(self, event):
        if self.bbox_start is not None:
            x0, y0 = self.bbox_start
            x1, y1 = (event.x, event.y)
            # Save the bounding box coordinates and class label
            bbox = ((x0, y0, x1, y1), self.selected_class)
            image_path = self.images[self.current_image_index]
            if image_path not in self.bbox_list:
                self.bbox_list[image_path] = []
            self.bbox_list[image_path].append(bbox)
            self.bbox_start = None

    def display_image(self):
        # Convert image from OpenCV BGR format to RGB format
        img_rgb = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
        # Resize image to fit into the canvas while maintaining aspect ratio
        img_height, img_width, _ = img_rgb.shape
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        if canvas_width / img_width < canvas_height / img_height:
            resize_factor = canvas_width / img_width
        else:
            resize_factor = canvas_height / img_height
        resized_img = cv2.resize(img_rgb, (int(img_width * resize_factor), int(img_height * resize_factor)))
        # Convert resized image to ImageTk format
        img_tk = ImageTk.PhotoImage(Image.fromarray(resized_img))
        # Display image on canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
        # Keep a reference to the image to prevent it from being garbage collected
        self.canvas.image = img_tk

    def save_annotations(self):
        if not self.bbox_list:
            messagebox.showwarning("No Annotations", "No annotations to save.")
            return

        save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if not save_path:
            return

        try:
            with open(save_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(["Image", "Class", "X_min", "Y_min", "X_max", "Y_max"])
                for image_path, annotations in self.bbox_list.items():
                    image_name = os.path.basename(image_path)
                    for bbox, cls in annotations:
                        x_min, y_min, x_max, y_max = bbox
                        csv_writer.writerow([image_name, cls, x_min, y_min, x_max, y_max])

            messagebox.showinfo("Annotations Saved", "Annotations saved successfully.")

        except Exception as e:
            messagebox.showerror("Error Saving Annotations", f"An error occurred while saving annotations: {str(e)}")

    def clear_annotations(self):
        self.bbox_list = {}  # Clear the bounding box list
        self.canvas.delete("bbox")  # Clear annotations displayed on the canvas

    def on_closing(self):
        # Save annotations when closing the application
        self.save_annotations()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotator(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)  # Bind on_closing method to close window event
    root.mainloop()


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import cv2
from PIL import Image, ImageTk
import os
import random
import csv


class ImageAnnotator:
    """
    A class for annotating images with bounding boxes.

    Attributes:
    - root: The root Tkinter window.
    - files_frame: The frame for the files section.
    - files_label: The label for the files section.
    - files_listbox: The listbox to display the image files.
    - load_button: The button for loading image files.
    - delete_button: The button for deleting image files.
    - class_frame: The frame for the classes section.
    - class_label: The label for the classes section.
    - class_listbox: The listbox to display the available classes.
    - add_class_button: The button for adding classes.
    - delete_class_button: The button for deleting classes.
    - save_button: The button for saving annotations.
    - images: A list of image file paths.
    - current_image_index: The index of the currently selected image.
    - current_image: The currently selected image.
    - bbox_start: The starting point of the bounding box.
    - bbox_list: A dictionary to store the bounding boxes for each image.
    - selected_class: The currently selected annotation class.
    - classes: A dictionary to store the annotation classes and their colors.
    - canvas: The canvas to display the image and annotations.
    - annotations_dict: A dictionary to store the annotations for each image.

    Methods:
    - load_images: Load image files.
    - delete_image: Delete an image file.
    - load_image: Load a selected image.
    - add_class: Add a new annotation class.
    - delete_class: Delete an annotation class.
    - select_class: Select an annotation class.
    - start_bbox: Start drawing a bounding box.
    - draw_bbox: Draw a bounding box.
    - end_bbox: End drawing a bounding box.
    - display_image: Display the current image on the canvas.
    - save_annotations: Save the annotations to a CSV file.
    - save_annotations_temp: Save the annotations temporarily before clearing the annotations dictionary.
    """
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotator")
        # Rest of the code...
class ImageAnnotator:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotator")

        # Create a frame for the files section
        self.files_frame = tk.Frame(root, bg="white")
        self.files_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create a label for the files section
        self.files_label = tk.Label(self.files_frame, text="Image Files", font=("Helvetica", 12), bg="white")
        self.files_label.pack(side=tk.TOP, pady=(0, 5))

        # Create a listbox to display the image files
        self.files_listbox = tk.Listbox(self.files_frame, width=40, height=20, bg="#f0f0f0")
        self.files_listbox.pack(side=tk.TOP, fill=tk.BOTH, padx=(0, 5), pady=(0, 5))
        self.files_listbox.bind('<<ListboxSelect>>', self.load_image)

        # Create buttons for loading and deleting image files
        self.load_button = tk.Button(self.files_frame, text="Load Images", command=self.load_images)
        self.load_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        self.delete_button = tk.Button(self.files_frame, text="Delete Image", command=self.delete_image)
        self.delete_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        # Create a frame for the classes section
        self.class_frame = tk.Frame(root, bg="white")
        self.class_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create a label for the classes section
        self.class_label = tk.Label(self.class_frame, text="Annotation Classes", font=("Helvetica", 12), bg="white")
        self.class_label.pack(side=tk.TOP, pady=(0, 5))

        # Create a listbox to display the available classes
        self.class_listbox = tk.Listbox(self.class_frame, width=20, height=10, bg="#f0f0f0")
        self.class_listbox.pack(side=tk.TOP, fill=tk.BOTH, padx=(0, 5), pady=(0, 5))
        self.class_listbox.bind('<<ListboxSelect>>', self.select_class)

        # Create buttons for adding and deleting classes
        self.add_class_button = tk.Button(self.class_frame, text="Add Class", command=self.add_class)
        self.add_class_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        self.delete_class_button = tk.Button(self.class_frame, text="Delete Class", command=self.delete_class)
        self.delete_class_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        # Create a button to save annotations
        self.save_button = tk.Button(root, text="Save Annotations", command=self.save_annotations)
        self.save_button.pack(side=tk.BOTTOM, pady=10)

        # Initialize attributes
        self.images = []
        self.current_image_index = None
        self.current_image = None
        self.bbox_start = None
        self.bbox_list = {}
        self.selected_class = None
        self.classes = {}

        # Create a canvas to display the image and annotations
        self.canvas = tk.Canvas(self.root, bg="white", highlightbackground="gray", highlightthickness=1)
        self.canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, padx=10, pady=10, expand=True)
        self.canvas.bind("<Button-1>", self.start_bbox)
        self.canvas.bind("<B1-Motion>", self.draw_bbox)
        self.canvas.bind("<ButtonRelease-1>", self.end_bbox)
        self.annotations_dict = {}

    def load_images(self):
        file_paths = filedialog.askopenfilenames(filetypes=[("Image files", "*.jpg; *.jpeg; *.png")])
        if file_paths:
            self.images = list(file_paths)
            self.files_listbox.delete(0, tk.END)  # Clear previous entries
            for image_path in self.images:
                self.files_listbox.insert(tk.END, os.path.basename(image_path))
            messagebox.showinfo("Images Loaded", "Image files loaded successfully.")
            self.clear_annotations()  # Clear annotations when loading new images

    def delete_image(self):
        selected_index = self.files_listbox.curselection()
        if selected_index:
            # Save annotations before clearing
            self.save_annotations_temp()

            del self.images[selected_index[0]]
            self.files_listbox.delete(selected_index[0])
            messagebox.showinfo("Image Deleted", "Image file deleted successfully.")

    def load_image(self, event):
        selected_index = self.files_listbox.curselection()
        if selected_index:
            # Save annotations before loading new image
            self.save_annotations_temp()

            self.current_image_index = selected_index[0]
            self.current_image = cv2.imread(self.images[self.current_image_index])
            self.display_image()

    def add_class(self):
        new_class = simpledialog.askstring("Add Class", "Enter new class name:")
        if new_class:
            self.class_listbox.insert(tk.END, new_class)
            self.classes[new_class] = '#' + "%06x" % random.randint(0, 0xFFFFFF)
            messagebox.showinfo("Class Added", "New class added successfully.")

    def delete_class(self):
        selected_index = self.class_listbox.curselection()
        if selected_index:
            selected_class = self.class_listbox.get(selected_index[0])
            del self.classes[selected_class]
            self.class_listbox.delete(selected_index[0])
            messagebox.showinfo("Class Deleted", "Class deleted successfully.")

    def select_class(self, event):
        selected_index = self.class_listbox.curselection()
        if selected_index:
            self.selected_class = self.class_listbox.get(selected_index[0])

    def start_bbox(self, event):
        if self.selected_class and self.current_image is not None:
            self.bbox_start = (event.x, event.y)

    def draw_bbox(self, event):
        if self.bbox_start is not None:
            x0, y0 = self.bbox_start
            x1, y1 = (event.x, event.y)
            self.canvas.delete("bbox")  # Delete previous bounding boxes
            # Draw all existing bounding boxes
            for bbox, cls in self.bbox_list.get(self.images[self.current_image_index], []):
                self.canvas.create_rectangle(*bbox, outline=self.classes[cls], tags="bbox")
            # Draw the current bounding box
            self.canvas.create_rectangle(x0, y0, x1, y1, outline=self.classes[self.selected_class], tags="bbox")

    def end_bbox(self, event):
        if self.bbox_start is not None:
            x0, y0 = self.bbox_start
            x1, y1 = (event.x, event.y)
            # Save the bounding box coordinates and class label
            bbox = ((x0, y0, x1, y1), self.selected_class)
            image_path = self.images[self.current_image_index]
            if image_path not in self.bbox_list:
                self.bbox_list[image_path] = []
            self.bbox_list[image_path].append(bbox)
            self.bbox_start = None

    def display_image(self):
        # Convert image from OpenCV BGR format to RGB format
        img_rgb = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
        # Resize image to fit into the canvas while maintaining aspect ratio
        img_height, img_width, _ = img_rgb.shape
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        if canvas_width / img_width < canvas_height / img_height:
            resize_factor = canvas_width / img_width
        else:
            resize_factor = canvas_height / img_height
        resized_img = cv2.resize(img_rgb, (int(img_width * resize_factor), int(img_height * resize_factor)))
        # Convert resized image to ImageTk format
        img_tk = ImageTk.PhotoImage(Image.fromarray(resized_img))
        # Display image on canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
        # Keep a reference to the image to prevent it from being garbage collected
        self.canvas.image = img_tk

    def save_annotations(self):
        # Save annotations before saving
        self.save_annotations_temp()

        if not self.annotations_dict:
            messagebox.showwarning("No Annotations", "No annotations to save.")
            return

        save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if not save_path:
            return

        try:
            with open(save_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(["Image", "Class", "X_min", "Y_min", "X_max", "Y_max"])
                for image_path, annotations in self.annotations_dict.items():
                    image_name = os.path.basename(image_path)
                    for bbox, cls in annotations:
                        x_min, y_min, x_max, y_max = bbox
                        csv_writer.writerow([image_name, cls, x_min, y_min, x_max, y_max])

            messagebox.showinfo("Annotations Saved", "Annotations saved successfully.")

        except Exception as e:
            messagebox.showerror("Error Saving Annotations", f"An error occurred while saving annotations: {str(e)}")

    def save_annotations_temp(self):
        """
        Save annotations temporarily before clearing the annotations dictionary.
        """
        if self.current_image_index is not None:
            image_path = self.images[self.current_image_index]
            if image_path not in self.annotations_dict:
                self.annotations_dict[image_path] = self.bbox_list.get(image_path, [])
            else:
                self.annotations_dict[image_path].extend(self.bbox_list.get(image_path, []))
            self.bbox_list.pop(image_path, None)  # Clear annotations for the current image

    def clear_annotations(self):
        self.bbox_list = {}  # Clear the bounding box list
        self.canvas.delete("bbox")  # Clear annotations displayed on the canvas
    
    def on_closing(self):
        # Save annotations when closing the application
        self.save_annotations()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotator(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)  # Bind on_closing method to close window event
    root.mainloop()


In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import cv2
from PIL import Image, ImageTk
import os
import random
import csv


class ImageAnnotator:
    """
    A class for annotating images with bounding boxes.

    Attributes:
    - root: The root Tkinter window.
    - files_frame: The frame for the files section.
    - files_label: The label for the files section.
    - files_listbox: The listbox to display the image files.
    - load_button: The button for loading image files.
    - delete_button: The button for deleting image files.
    - class_frame: The frame for the classes section.
    - class_label: The label for the classes section.
    - class_listbox: The listbox to display the available classes.
    - add_class_button: The button for adding classes.
    - delete_class_button: The button for deleting classes.
    - save_button: The button for saving annotations.
    - images: A list of image file paths.
    - current_image_index: The index of the currently selected image.
    - current_image: The currently selected image.
    - bbox_start: The starting point of the bounding box.
    - bbox_list: A dictionary to store the bounding boxes for each image.
    - selected_class: The currently selected annotation class.
    - classes: A dictionary to store the annotation classes and their colors.
    - canvas: The canvas to display the image and annotations.
    - annotations_dict: A dictionary to store the annotations for each image.

    Methods:
    - load_images: Load image files.
    - delete_image: Delete an image file.
    - load_image: Load a selected image.
    - add_class: Add a new annotation class.
    - delete_class: Delete an annotation class.
    - select_class: Select an annotation class.
    - start_bbox: Start drawing a bounding box.
    - draw_bbox: Draw a bounding box.
    - end_bbox: End drawing a bounding box.
    - display_image: Display the current image on the canvas.
    - save_annotations: Save the annotations to a CSV file.
    - save_annotations_temp: Save the annotations temporarily before clearing the annotations dictionary.
    """
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotator")
        # Rest of the code...
class ImageAnnotator:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotator")

        # Create a frame for the files section
        self.files_frame = tk.Frame(root, bg="white")
        self.files_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create a label for the files section
        self.files_label = tk.Label(self.files_frame, text="Image Files", font=("Helvetica", 12), bg="white")
        self.files_label.pack(side=tk.TOP, pady=(0, 5))

        # Create a listbox to display the image files
        self.files_listbox = tk.Listbox(self.files_frame, width=40, height=20, bg="#f0f0f0")
        self.files_listbox.pack(side=tk.TOP, fill=tk.BOTH, padx=(0, 5), pady=(0, 5))
        self.files_listbox.bind('<<ListboxSelect>>', self.load_image)

        # Create buttons for loading and deleting image files
        self.load_button = tk.Button(self.files_frame, text="Load Images", command=self.load_images)
        self.load_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        self.delete_button = tk.Button(self.files_frame, text="Delete Image", command=self.delete_image)
        self.delete_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        # Create a frame for the classes section
        self.class_frame = tk.Frame(root, bg="white")
        self.class_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create a label for the classes section
        self.class_label = tk.Label(self.class_frame, text="Annotation Classes", font=("Helvetica", 12), bg="white")
        self.class_label.pack(side=tk.TOP, pady=(0, 5))

        # Create a listbox to display the available classes
        self.class_listbox = tk.Listbox(self.class_frame, width=20, height=10, bg="#f0f0f0")
        self.class_listbox.pack(side=tk.TOP, fill=tk.BOTH, padx=(0, 5), pady=(0, 5))
        self.class_listbox.bind('<<ListboxSelect>>', self.select_class)

        # Create buttons for adding and deleting classes
        self.add_class_button = tk.Button(self.class_frame, text="Add Class", command=self.add_class)
        self.add_class_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        self.delete_class_button = tk.Button(self.class_frame, text="Delete Class", command=self.delete_class)
        self.delete_class_button.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)

        # Create a button to save annotations
        self.save_button = tk.Button(root, text="Save Annotations", command=self.save_annotations)
        self.save_button.pack(side=tk.BOTTOM, pady=10)
        
        #create a button to clear annotations for the current image
        self.clear_button = tk.Button(root, text="Clear Annotations", command=self.clear_annotations_for_image)
        self.clear_button.pack(side=tk.BOTTOM, pady=10)

        # Initialize attributes
        self.images = []
        self.current_image_index = None
        self.current_image = None
        self.bbox_start = None
        self.bbox_list = {}
        self.selected_class = None
        self.classes = {}

        # Create a canvas to display the image and annotations
        self.canvas = tk.Canvas(self.root, bg="white", highlightbackground="gray", highlightthickness=1)
        self.canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, padx=10, pady=10, expand=True)
        self.canvas.bind("<Button-1>", self.start_bbox)
        self.canvas.bind("<B1-Motion>", self.draw_bbox)
        self.canvas.bind("<ButtonRelease-1>", self.end_bbox)
        self.annotations_dict = {}

    def load_images(self):
        file_paths = filedialog.askopenfilenames(filetypes=[("Image files", "*.jpg; *.jpeg; *.png")])
        if file_paths:
            self.images = list(file_paths)
            self.files_listbox.delete(0, tk.END)  # Clear previous entries
            for image_path in self.images:
                self.files_listbox.insert(tk.END, os.path.basename(image_path))
            messagebox.showinfo("Images Loaded", "Image files loaded successfully.")
            self.clear_annotations()  # Clear annotations when loading new images

    def delete_image(self):
        selected_index = self.files_listbox.curselection()
        if selected_index:
            # Save annotations before clearing
            self.save_annotations_temp()

            del self.images[selected_index[0]]
            self.files_listbox.delete(selected_index[0])
            messagebox.showinfo("Image Deleted", "Image file deleted successfully.")

    def load_image(self, event):
        selected_index = self.files_listbox.curselection()
        if selected_index:
            # Save annotations before loading new image
            self.save_annotations_temp()

            self.current_image_index = selected_index[0]
            self.current_image = cv2.imread(self.images[self.current_image_index])
            self.display_image()

            # Display annotations if available for the loaded image
            image_path = self.images[self.current_image_index]
            if image_path in self.annotations_dict:
                for bbox, cls in self.annotations_dict[image_path]:
                    self.canvas.create_rectangle(*bbox, outline=self.classes[cls], tags="bbox")


    def add_class(self):
        new_class = simpledialog.askstring("Add Class", "Enter new class name:")
        if new_class:
            self.class_listbox.insert(tk.END, new_class)
            self.classes[new_class] = '#' + "%06x" % random.randint(0, 0xFFFFFF)
            messagebox.showinfo("Class Added", "New class added successfully.")

    def delete_class(self):
        selected_index = self.class_listbox.curselection()
        if selected_index:
            selected_class = self.class_listbox.get(selected_index[0])
            del self.classes[selected_class]
            self.class_listbox.delete(selected_index[0])
            messagebox.showinfo("Class Deleted", "Class deleted successfully.")

    def select_class(self, event):
        selected_index = self.class_listbox.curselection()
        if selected_index:
            self.selected_class = self.class_listbox.get(selected_index[0])

    def start_bbox(self, event):
        if self.selected_class and self.current_image is not None:
            self.bbox_start = (event.x, event.y)

    def draw_bbox(self, event):
        if self.bbox_start is not None:
            x0, y0 = self.bbox_start
            x1, y1 = (event.x, event.y)
            self.canvas.delete("bbox")  # Delete previous bounding boxes
            # Draw all existing bounding boxes
            for bbox, cls in self.bbox_list.get(self.images[self.current_image_index], []):
                self.canvas.create_rectangle(*bbox, outline=self.classes[cls], tags="bbox")
            # Draw the current bounding box
            self.canvas.create_rectangle(x0, y0, x1, y1, outline=self.classes[self.selected_class], tags="bbox")

    def end_bbox(self, event):
        if self.bbox_start is not None:
            x0, y0 = self.bbox_start
            x1, y1 = (event.x, event.y)
            # Save the bounding box coordinates and class label
            bbox = ((x0, y0, x1, y1), self.selected_class)
            image_path = self.images[self.current_image_index]
            if image_path not in self.bbox_list:
                self.bbox_list[image_path] = []
            self.bbox_list[image_path].append(bbox)
            self.bbox_start = None

    def display_image(self):
        # Convert image from OpenCV BGR format to RGB format
        img_rgb = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
        # Resize image to fit into the canvas while maintaining aspect ratio
        img_height, img_width, _ = img_rgb.shape
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        if canvas_width / img_width < canvas_height / img_height:
            resize_factor = canvas_width / img_width
        else:
            resize_factor = canvas_height / img_height
        resized_img = cv2.resize(img_rgb, (int(img_width * resize_factor), int(img_height * resize_factor)))
        # Convert resized image to ImageTk format
        img_tk = ImageTk.PhotoImage(Image.fromarray(resized_img))
        # Display image on canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
        # Keep a reference to the image to prevent it from being garbage collected
        self.canvas.image = img_tk

    def save_annotations(self):
        # Save annotations before saving
        self.save_annotations_temp()

        if not self.annotations_dict:
            messagebox.showwarning("No Annotations", "No annotations to save.")
            return

        save_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if not save_path:
            return

        try:
            with open(save_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(["Image", "Class", "X_min", "Y_min", "X_max", "Y_max"])
                for image_path, annotations in self.annotations_dict.items():
                    image_name = os.path.basename(image_path)
                    for bbox, cls in annotations:
                        x_min, y_min, x_max, y_max = bbox
                        csv_writer.writerow([image_name, cls, x_min, y_min, x_max, y_max])

            messagebox.showinfo("Annotations Saved", "Annotations saved successfully.")

        except Exception as e:
            messagebox.showerror("Error Saving Annotations", f"An error occurred while saving annotations: {str(e)}")

    def save_annotations_temp(self):
        """
        Save annotations temporarily before clearing the annotations dictionary.
        """
        if self.current_image_index is not None:
            image_path = self.images[self.current_image_index]
            if image_path not in self.annotations_dict:
                self.annotations_dict[image_path] = self.bbox_list.get(image_path, [])
            else:
                self.annotations_dict[image_path].extend(self.bbox_list.get(image_path, []))
            self.bbox_list.pop(image_path, None)  # Clear annotations for the current image

    def clear_annotations(self):
        self.bbox_list = {}  # Clear the bounding box list
        self.canvas.delete("bbox")  # Clear annotations displayed on the canvas

    def clear_annotations_for_image(self):
        if self.current_image_index is not None:
            # Clear annotations for the current image
            image_path = self.images[self.current_image_index]
            self.bbox_list.pop(image_path, None)
            self.canvas.delete("bbox")

            # Optionally, you can also reset the selected class
            self.selected_class = None

            # Show a message informing the user that annotations are cleared
            messagebox.showinfo("Annotations Cleared", "Annotations for the current image cleared. You can now redraw annotations.")

            # You may also want to reset any other relevant attributes or UI elements
            # For example, if you want to allow users to select a new class for annotations:
            self.class_listbox.selection_clear(0, tk.END)
            self.selected_class = None

    
    def on_closing(self):
        # Save annotations when closing the application
        self.save_annotations()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotator(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)  # Bind on_closing method to close window event
    root.mainloop()
